<a href="https://colab.research.google.com/github/theadarshkr/Feynn-Labs-Assignment/blob/main/CropDiseasePred%26Treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_score
from keras.models import load_model
import cv2
import matplotlib.pyplot as plt

#Loading the data
data = pd.read_csv('data.csv')

#Creating TF-IDF vectorizers for symptoms and treatments
symptoms_vectorizer = TfidfVectorizer()
treatments_vectorizer = TfidfVectorizer()

#Converting the symptoms and treatments to numerical vectors
symptoms_vectors = symptoms_vectorizer.fit_transform(data['symptoms'])
treatments_vectors = treatments_vectorizer.fit_transform(data['treatments'])

#Creating a numerical vector for each disease using the symptoms vectorizer
disease_vectors = symptoms_vectorizer.transform(data['disease_name'])

#Loading the CNN model
model = load_model('cnn_model.h5')

#Defining a function to predict the disease using SVM
def predict_disease_svm(symptoms):
    #Converting the symptoms to a numerical vector
    symptoms_vector = symptoms_vectorizer.transform([symptoms])

    #Calculating the cosine similarity between the symptoms vector and the disease vectors
    similarity_scores = symptoms_vector @ disease_vectors.T #Using matrix multiplication instead of cosine_similarity

    #Getting the index of the most similar disease
    most_similar_disease_index = np.argmax(similarity_scores)

    #Returning the name of the most similar disease
    return data['disease_name'][most_similar_disease_index]

#Defining a function to predict the disease using CNN
def predict_disease_cnn(image):
    #Preprocessing the image
    image = cv2.resize(image, (224, 224))
    image = np.array(image) / 255.0

    #Predicting the disease using the CNN model
    prediction = model.predict(np.expand_dims(image, axis=0))

    #Returning the name of the predicted disease
    return data['disease_name'][np.argmax(prediction)]

#Defining a function to recommend treatments
def recommend_treatments(disease):
    #Getting the list of recommended treatments for the disease
    treatments = data[data['disease_name'] == disease]['treatment'].tolist()

    #Converting the treatments to numerical vectors
    treatment_vectors = treatments_vectorizer.transform(treatments)

    #Calculating the Jaccard similarity between the treatment vectors and the disease vector
    similarity_scores = np.apply_along_axis(jaccard_score, 1, treatment_vectors.toarray(), disease_vectors.toarray()) #Using np.apply_along_axis instead of a for loop

    #Getting the indices of the most similar treatments
    most_similar_treatment_indices = np.argpartition(similarity_scores, -5)[-5:]

    #Returning the list of most similar treatments
    return np.take_along_axis(data['treatment'], most_similar_treatment_indices, axis=0) #Using np.take_along_axis instead of a list comprehension

#User input
symptoms = input("Enter the symptoms of the crop disease: ")
image_path = input("Enter the path to the image of the affected crop: ")

#Predicting the disease using SVM and CNN
predicted_disease_svm = predict_disease_svm(symptoms)
predicted_disease_cnn = predict_disease_cnn(cv2.imread(image_path))

#Printing the predicted diseases
print("Predicted disease using SVM:", predicted_disease_svm)
print("Predicted disease using CNN:", predicted_disease_cnn)

#Recommending treatments for the predicted disease
treatments = recommend_treatments(predicted_disease_cnn)

#Printing the recommended treatments
print("Recommended treatments:")
for treatment in treatments:
    print("-", treatment)

#Displaying the image to the user
plt.imshow(cv2.imread(image_path))
plt.show()

#Displaying the model architecture to the user
model.summary()